<a href="https://colab.research.google.com/github/kangjunseo/Bioinformatics/blob/main/Novozymes_Enzyme_Stability_Prediction_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install pytorch-lightning

1.13.0+cu116
     |████████████████████████████████| 9.4 MB 26.6 MB/s 
     |████████████████████████████████| 4.5 MB 30.3 MB/s 
     |████████████████████████████████| 512 kB 11.7 MB/s 
     |████████████████████████████████| 280 kB 69.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 800 kB 33.6 MB/s 
     |████████████████████████████████| 125 kB 55.4 MB/s 


In [ ]:
import torch_geometric
use_cuda_if_available = True

# Data preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd, numpy as np, nltk
WT = "VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK"
test_df = pd.read_csv("gdrive/MyDrive/input/novozymes-enzyme-stability-prediction/test.csv")
test_df_extra = pd.read_csv("gdrive/MyDrive/input/novozymes-enzyme-stability-prediction/new_test.csv")
test_df_extra.head()

,seq_id,mutant_sequence,pH,data_source,data_source_enum,n_edits,edit_type,edit_idx,wildtype_aa,mutant_aa,wildtype_sequence,wildtype_pt_path,mutant_pt_path
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,-1,1,substitution,16,L,E,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,/kaggle/input/esm2-firetraintest-embeddings-pe...,/kaggle/input/esm2-firetraintest-embeddings-pe...
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,-1,1,substitution,16,L,K,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,/kaggle/input/esm2-firetraintest-embeddings-pe...,/kaggle/input/esm2-firetraintest-embeddings-pe...
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,-1,1,deletion,16,L,NaN,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,/kaggle/input/esm2-firetraintest-embeddings-pe...,/kaggle/input/esm2-firetraintest-embeddings-pe...
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,-1,1,substitution,17,K,C,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,/kaggle/input/esm2-firetraintest-embeddings-pe...,/kaggle/input/esm2-firetraintest-embeddings-pe...
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,-1,1,substitution,17,K,F,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,/kaggle/input/esm2-firetraintest-embeddings-pe...,/kaggle/input/esm2-firetraintest-embeddings-pe...


In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.1 MB 35.2 MB/s 


In [ ]:
from Bio import PDB

def get_normalized_vector_from_atoms(atoms):  
    list_atoms = list(atoms)
    _sum = sum([i.get_vector().normalized() for i in list_atoms], start = PDB.vectors.Vector(0,0,0))
    return _sum / len(list_atoms)

def get_normalized_vectors_from_PDB(pdbPath, pdbcode):
    pdbparser = PDB.PDBParser(QUIET=True)   # suppress PDBConstructionWarning
    struct = pdbparser.get_structure(pdbcode, pdbPath)
    residue = list(next(next(struct.get_models()).get_chains()).get_residues()) #struct -> model -> chain -> residue (only 1 model, 1 chain)
    print(residue)
    return [get_normalized_vector_from_atoms(i.get_atoms()) for i in residue]

In [ ]:
pdbPath = "/content/gdrive/MyDrive/input/novozymes-enzyme-stability-prediction/wildtype_structure_prediction_af2.pdb"
pdbcode = 'AF2'
vectors = get_normalized_vectors_from_PDB(pdbPath, pdbcode)
print(vectors)
print(len(vectors))

[<Residue VAL het=  resseq=1 icode= >, <Residue PRO het=  resseq=2 icode= >, <Residue VAL het=  resseq=3 icode= >, <Residue ASN het=  resseq=4 icode= >, <Residue PRO het=  resseq=5 icode= >, <Residue GLU het=  resseq=6 icode= >, <Residue PRO het=  resseq=7 icode= >, <Residue ASP het=  resseq=8 icode= >, <Residue ALA het=  resseq=9 icode= >, <Residue THR het=  resseq=10 icode= >, <Residue SER het=  resseq=11 icode= >, <Residue VAL het=  resseq=12 icode= >, <Residue GLU het=  resseq=13 icode= >, <Residue ASN het=  resseq=14 icode= >, <Residue VAL het=  resseq=15 icode= >, <Residue ALA het=  resseq=16 icode= >, <Residue LEU het=  resseq=17 icode= >, <Residue LYS het=  resseq=18 icode= >, <Residue THR het=  resseq=19 icode= >, <Residue GLY het=  resseq=20 icode= >, <Residue SER het=  resseq=21 icode= >, <Residue GLY het=  resseq=22 icode= >, <Residue ASP het=  resseq=23 icode= >, <Residue SER het=  resseq=24 icode= >, <Residue GLN het=  resseq=25 icode= >, <Residue SER het=  resseq=26 icod

# Preparing Input data

In [ ]:
from torch.utils.data import dataloader
from pytorch_lightning import LightningDataModule
from torch_geometric.data import Data, Batch
from torch.nn.functional import pad

In [ ]:
class CustomData(LightningDataModule):
    def __init__(self, training_set, validation_set, test_set, batch_size=128, num_workers=1):
        super().__init__()
        self.training_set = training_set
        self.validation_set = validation_set
        self.test_set = test_set
        self.batch_size = batch_size
        self.num_workers = num_workers

    def collate_function(self, batch):
        return Batch.from_data_list([Data(edge_index=data.edge_index,
                                          x=pad(data.x, (0,3), 'constant', 0.)[:,:53],
                                          edge_attr=data.edge_attr,
                                          y=data.y.unsqueeze(0).float()) for data in batch])
        
    def train_dataloader(self): 
        return dataloader(self.training_set, batch_size=self.batch_size, num_workers=self.num_workers, collate_fn=self.collate_function)
        
    def val_dataloader(self): 
        return dataloader(self.validation_set, batch_size=self.batch_size, num_workers=self.num_workers, collate_fn=self.collate_function)
    
    def test_dataloader(self): 
        return dataloader(self.test_set, batch_size=self.batch_size, num_workers=self.num_workers, collate_fn=self.collate_function)
        

In [ ]:
class DistogramAttention(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.linear = nn.Linear(input_dim, output_dim):
  
    def forward(self, distogram, features):
        features = self.linear(features) # dimension of features is (n_of_seq * embedding_size)
        features = distogram.matmul(features) # dimension of distogram is (n_of_seq * n_of_seq)
        return features

In [ ]:
from torch.nn import Module, Linear
from torch_geometric.nn import GCNConv, global_mean_pool

### 여기서 GCNConv 대신 직접 짠 모델로 바꾸는 방향

class CustomGCN(Module):
    def __init__(self, input_size, label_size, layer_size=64, dropout=0.1, heads=2):
        super().__init__()
        self.input_size = input_size
        self.label_size = label_size
        self.layer_size = layer_size
        self.dropout = dropout
        self.heads = heads
        self.setup()

    def setup(self):
        self.first_layer = GCNConv(self.input_size, self.layer_size, dropout=self.dropout, heads=self.heads, concat=False)
        self.last_layer = GCNConv(self.layer_size, self.layer_size, dropout=self.dropout, heads=self.heads, concat=False)
        self.ffnn = Linear(self.layer_size, self.label_size)

    def convert_graph_into_single_vector(self, graph_hidden, batch_index):
        return global_mean_pool(graph_hidden, batch_index)

    def forward(self, batch):
        z = self.first_layer(batch.x, batch.edge_index)
        z = self.last_layer(z, batch.edge_index)
        z = self.convert_graph_into_single_vector(z, batch.batch)
        z = self.ffnn(z)
        return z

In [ ]:
from pytorch_lightning import LightningModule
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam

class CustomModel(LightningModule):
    def __init__(self, model, learning_rate=1e-3):
        super().__init__()
        self.model = model
        self.lr = learning_rate

    def forward(self, batch, mode):
        z = self.model(batch)
        loss = self.loss_function(z, batch.y)
        self.log(f"{mode}_loss", loss, batch_size=batch.y.size(0), prog_bar=True, on_step=False, on_epoch=True)
        return loss, z, batch.y

    def training_step(self, batch, batch_idx):
        loss, predict, answer = self(batch, 'train')
        return {'loss':loss, 'predict':predict, 'answer':answer}

    def validation_step(self, batch, batch_idx):
        loss, predict, answer = self(batch, 'val')
        return {'loss':loss, 'predict':predict, 'answer':answer}

    def test_step(self, batch, batch_idx):
        loss, predict, answer = self(batch, 'test')
        return {'loss':loss, 'predict':predict, 'answer':answer}

    def predict_step(self, batch, batch_idx):
        predict = self.model(batch)
        return predict

    def loss_function(self, output, target):
        return BCEWithLogitsLoss(reduction='mean')(output, target)

    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=self.lr)
        return optimizer
     

In [ ]:
from pytorch_lightning import Trainer

import warnings
warnings.filterwarnings(action='ignore')

# training_dataset = 
# validation_dataset = 
# test_dataset = 

data_module = CustomData(training_dataset, validation_dataset, test_dataset)

gat = CustomGCN(53, 1)
model = CustomModel(gat)

trainer = Trainer(max_epochs=1, accelerator='gpu', devices=[0])
trainer.fit(model, datamodule=data_module)

NameError: ignored

In [ ]:
from torchmetrics.functional import auroc

outputs = trainer.predict(model, dataloaders=data_module.test_dataloader())
y = torch.concat(outputs)
x = torch.concat([batch.y for batch in data_module.test_dataloader()]).int()

evaluation = auroc(y, x)
print(f"auc-roc: {evaluation}")